# 加上两个用户行为特征

1. 8234 base
2. n_estimators 91
3. gamma 6.6

In [2]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline

train_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt',sep=' ')
test_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt',sep=' ')

# 线下线上数据统一进行特征处理
test_df['is_trade'] = -1
total_df = pd.concat([train_df, test_df], axis=0, ignore_index=True)

# 1 先处理时序数据

In [3]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数
# date最终不使用，直接用day(第 0 - 7 天)

from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

total_df['date'] = total_df['context_timestamp'].apply(lambda x: extract_date(x))
total_df['day'] = le.fit_transform(total_df['date'])
total_df['weekday'] = total_df['context_timestamp'].apply(lambda x: extract_weekday(x))
total_df['hour'] = total_df['context_timestamp'].apply(lambda x: extract_hour(x))

# 2 先分离训练集, 测试集, 线上集



In [4]:
# 获取训练测试的索引, 6全集数据验证, 7为生成上线文件
import numpy as np
test_day= 6
starts = list(range(0,test_day))
ends = [test_day]
train_indices = total_df[total_df['day'].isin(starts)].index.values
test_indices = total_df[total_df['day'].isin(ends)].index.values
print(train_indices.shape, test_indices.shape)

tmp_df = total_df.copy()

# 把测试索引的label提取
y_test = tmp_df.iloc[test_indices]['is_trade']

# 把需要训练和测试的数据提取, 并且强制去掉测试集的label列
tmp_df.loc[test_indices, 'is_trade'] = np.nan

# 得出训练测试必须的数据集 ,并且添加一列data_set作为标记
train_tmp = tmp_df.iloc[train_indices].copy()
train_tmp['data_set'] = 'training'
test_tmp = tmp_df.iloc[test_indices].copy()
test_tmp['data_set'] = 'testing'
raw_df = train_tmp.append(test_tmp)


(420717,) (57421,)


### 重复列特征

In [5]:
%run ../util/time_utils.py
dup_feat = ['item_id', 'item_brand_id', 'shop_id', 'user_id']
raw_df = generateColDupByDay(raw_df, dup_feat, list(range(1, 8)), verbose=False)

### 交易率特征选择

*固定smooth为10*

1. 特征是否独立提升?
2. 平滑是否不会改变正负向?

##### 正向特征

> item_city_id, shop_id, user_gender_id, item_sales_level, item_collected_level, shop_review_num_level

##### 负向特征

> item_price_level,
item_id,item_brand_id,item_pv_level,user_age_level,
user_occupation_id,user_star_level,context_page_id,
shop_review_positive_rate,shop_star_level,
shop_score_service,shop_score_description,day,hour


In [6]:
# 计算前一天的交易率set到下一天，第0天用回自己
# %run ../util/time_utils.py

# setTradeRateByDate(raw_df, ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id'])

# 计算前一天的交易率set到下一天，第0天用回自己
%run ../util/trade_info.py

# trade_rela = ['item_city_id', 'shop_id', 'user_gender_id', 'item_sales_level', 'item_collected_level', 'shop_review_num_level']
trade_rela = ['item_city_id', 'item_id', 'item_brand_id', 'shop_id', 'user_id']

# colSm = {}
# for col in trade_rela:
#     colSm[col] = [15*(mean0) , 15]
generateTradeRateByDate(raw_df, trade_rela, 7, None, verbose=False, glbSmoothing=200, glbMean0=0.05)
print(raw_df.shape)

C:\Anaconda3\lib\site-packages\pandas\core\series.py:696: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]


(478138, 71)


### 复合类型拆解

In [7]:
%run ../util/complex_type.py
from sklearn import preprocessing
from datetime import datetime
le = preprocessing.LabelEncoder()
raw_df = process_complex_types(raw_df, get_icl_map(raw_df), get_ipl_map(raw_df))

raw_df['item_category_1'] = le.fit_transform(raw_df.item_category_1)
raw_df['item_category_2'] = le.fit_transform(raw_df.item_category_2)

get_icl_map ... 
get_ipl_map ... 
processing predict_category_property ...
processing item_property_list ...
processing item_category_list ...
generating item_category_1, item_category_2 ...


In [8]:
%run ../util/complex_type.py

cnt_user_item_review = raw_df[["user_id", "item_id", "instance_id"]].groupby(["user_id", "item_id"])['instance_id'].count().to_dict() 
cnt_user_cate_review = raw_df[["user_id", "item_category_1", "instance_id"]].groupby(["user_id", "item_category_1"])['instance_id'].count().to_dict()  

f1 = set_review_cnt("user_id", "item_id", cnt_user_item_review)
f2 = set_review_cnt("user_id", "item_category_1", cnt_user_cate_review)

tmp = raw_df.sort_values(by="context_timestamp")
tmp["item_review_cnt"] = tmp[["user_id", "item_id"]].apply(f1, axis=1)
tmp["cate_review_cnt"] = tmp[["user_id", "item_category_1"]].apply(f2, axis=1)
raw_df = tmp.sort_index()


In [ ]:
raw_df['cate_review_cnt']

In [16]:
set(a)

{0}

# 3 生成矩阵数据

In [20]:
train_df = raw_df[raw_df['data_set'] == 'training']
test_df = raw_df[raw_df['data_set'] == 'testing']

non_feat_columns = ['data_set', 'context_timestamp', 'instance_id', 'is_trade', 'context_id',
                   'item_property_list', 'item_category_list', 'date', 'predict_category_property',
                    'predict_richness', 'predict_category_property', 'item_property_richness', 'item_property_list', 
                    'item_category_list', 'item_category_1', 'item_category_2'
                   ]

D = train_df[['date']]
X_train = train_df.drop(non_feat_columns, axis=1)
y_train = train_df[['is_trade']].values.ravel()
X_test = test_df.drop(non_feat_columns, axis=1)
# y_test is already exists

# X_online = test_df.drop(non_feat_columns, axis=1).values
print(D.shape, X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(420717, 1) (420717, 64) (420717,) (57421, 64) (57421,)


In [22]:
raw_df['user_age_level'].unique()

array([1003, 1002, 1004, 1006, 1005,   -1, 1007, 1000, 1001], dtype=int64)

In [21]:
# 训练模型
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost

#import lightgbm as lgb

%run ../util/time_series_split.py

# clf = lgb.LGBMClassifier(n_jobs=20)
clf = xgboost.XGBClassifier(n_jobs=7,max_depth=5,n_estimators=91,min_child_weight=5, gamma=6.6)
# clf = xgboost.XGBClassifier(n_jobs=7, max_depth=3, n_estimators=230)

m = None

# 如果移动到线上集, 则输出模型
if sum(y_test == -1) > 0:
    m = clf.fit(X_train, y_train)
    result = pd.DataFrame()
    result['instance_id'] = test_df['instance_id']
    result['predicted_score'] = pd.DataFrame(m.predict_proba(X_test))[1].values
    result.to_csv('submits/9_7416_8097_a8177_b8063.csv', sep = ' ', header=True, index = False)
else:
    # 分离a,b榜
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=6)
    m = clf.fit(X_train, y_train)
    
    val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
    val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
    print('(%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
          (starts, ends, \
           log_loss(y_train, m.predict_proba(X_train)), \
           log_loss(y_test, m.predict_proba(X_test)),\
          val_a_loss, val_b_loss))
    
# ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07348, test logloss: 0.08093, a: 0.08162, b: 0.08064
# ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07379, test logloss: 0.08092, a: 0.08160, b: 0.08063
# ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07424, test logloss: 0.08098, a: 0.08179, b: 0.08064
# ([0, 1, 2, 3, 4, 5] -> [6]) gamma6.6 train logloss: 0.07416, test logloss: 0.08097, a: 0.08177, b: 0.08063
# ([0, 1, 2, 3, 4, 5] -> [6]) +max_delta_step 0.9 train logloss: 0.07452, test logloss: 0.08095, a: 0.08170, b: 0.08063

([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08177, b: 0.08063


In [25]:
m = clf.fit(X_train, y_train)
for i in range(1000):
    # 分离a,b榜
    X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=i)
    
    val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
    val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
    print('%d (%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
          (i, starts, ends, \
           log_loss(y_train, m.predict_proba(X_train)), \
           log_loss(y_test, m.predict_proba(X_test)),\
          val_a_loss, val_b_loss))

0 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08475, b: 0.07936
1 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07688, b: 0.08273
2 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08384, b: 0.07975
3 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08053, b: 0.08117
4 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07782, b: 0.08233
5 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08451, b: 0.07946
6 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08177, b: 0.08063
7 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08515, b: 0.07918
8 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07797, b: 0.08226
9 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08517, b: 0.07918


82 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08020, b: 0.08130
83 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08394, b: 0.07970
84 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08015, b: 0.08133
85 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08003, b: 0.08138
86 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07672, b: 0.08280
87 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08490, b: 0.07929
88 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08389, b: 0.07972
89 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07770, b: 0.08238
90 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08075, b: 0.08107
91 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07732, b

163 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07565, b: 0.08326
164 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08413, b: 0.07962
165 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08112, b: 0.08091
166 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07942, b: 0.08164
167 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08254, b: 0.08030
168 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08191, b: 0.08057
169 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08117, b: 0.08089
170 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07952, b: 0.08159
171 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08390, b: 0.07972
172 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 

244 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08154, b: 0.08073
245 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07739, b: 0.08251
246 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08259, b: 0.08028
247 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07946, b: 0.08162
248 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07825, b: 0.08214
249 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08639, b: 0.07865
250 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08082, b: 0.08104
251 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08049, b: 0.08118
252 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08315, b: 0.08004
253 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 

325 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07671, b: 0.08280
326 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08381, b: 0.07976
327 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08306, b: 0.08008
328 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07955, b: 0.08158
329 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08128, b: 0.08084
330 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07944, b: 0.08163
331 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08230, b: 0.08040
332 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08208, b: 0.08050
333 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07929, b: 0.08169
334 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 

406 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08479, b: 0.07934
407 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08230, b: 0.08041
408 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08351, b: 0.07989
409 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.08051, b: 0.08117
410 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07675, b: 0.08278
411 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07753, b: 0.08245
412 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07983, b: 0.08146
413 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07982, b: 0.08147
414 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 0.07962, b: 0.08156
415 ([0, 1, 2, 3, 4, 5] -> [6]) train logloss: 0.07416, test logloss: 0.08097, a: 

KeyboardInterrupt: 

# 4 超参搜索

In [ ]:
from sklearn.model_selection import ParameterGrid
# 训练模型
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import xgboost

params={
    'max_depth': [5],#[6,10,15,20], # 5 is good but takes too long in kaggle env
    'subsample': [1],#[1,0.9,0.8,0.7],#[0.1,0.2,0.3,0.4,0.5,0.55,0.6,0.65,0.7,0.8,0.9],#[0.9]
    'colsample_bytree': [1],#[0.1,0.2,0.3,0.4,0.5,0.6,0.65,0.7,0.75,0.8,0.9,0.95],#[0.9],
    'colsample_bylevel':[1],#[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1],
    'scale_pos_weight':[1],#[0.7,0.8,1,1.1,1.2],#0.1,0.5,1,2,5
    'max_delta_step':[0.9],#[0,0.1,0.001,0.05,0.005,0.5,1,2],
    'n_estimators': [91],#[100,90,91,92,93,94,95,96,97,98,99],#[200,230,260,270,280,290,300]
    'reg_lambda': [1],#[1.3,1.4,1.5,1.6],
    'reg_alpha': [0],#[0,0.01,0.1,0.02,0.2],#[0.01,0.05,0.005,0.2,0.1,0.02,0.0001,0],#
    'min_child_weight':[5],#[7,8,9,10,11,12,13,14,15,16,17,18,19,20],#[23],#[20,21,22,23,24,25,26],#
    'gamma':[6.6],#[6],#[0.1],#
    'learning_rate':[0.1],#[0.01,0.015,0.02,0.025,0.05,0.005,0.1],#[0.02]#
}

# if sum(y_test == -1) == 0:
#     xgb = xgboost.XGBClassifier(n_jobs=7)
#     best_score = 1 
#     for g in ParameterGrid(params):
#         xgb.set_params(**g)

#         # 分离a,b榜
#         X_val_a, X_val_b, y_val_a, y_val_b = train_test_split(X_test, y_test, test_size=0.7, shuffle=True, random_state=6)
#         m = xgb.fit(X_train, y_train, eval_metric='logloss')

#         val_train_loss = log_loss(y_train, m.predict_proba(X_train))
#         val_test_loss = log_loss(y_test, m.predict_proba(X_test))
#         val_a_loss = log_loss(y_val_a, m.predict_proba(X_val_a))
#         val_b_loss = log_loss(y_val_b, m.predict_proba(X_val_b))
#         print('-'*80)
#         print(g)
#         print('(%s -> %s) train logloss: %.5f, test logloss: %.5f, a: %.5f, b: %.5f' % \
#               (starts, ends, val_train_loss, val_test_loss, val_a_loss, val_b_loss))

#         # save if best
#         if val_test_loss < best_score:
#             best_score = val_test_loss
#             best_grid = g

#     print('-'*80, '\n')        
#     print ("log loss: %0.5f" % best_score )
#     print ("Grid:", best_grid)

In [ ]:
from datetime import datetime
print(datetime.now())